# Imports + config

In [ ]:
import os
import numpy as np
import pandas as pd

from sqlalchemy import create_engine, text

import mlflow
import mlflow.sklearn
import mlflow.lightgbm

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

import lightgbm as lgb
import shap
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)

# MLflow inside docker network
mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("cewas_telco_churn_lgbm")

# Postgres engine
PGUSER = os.getenv("POSTGRES_USER", "cewas")
PGPASS = os.getenv("POSTGRES_PASSWORD", "cewas")
PGDB   = os.getenv("POSTGRES_DB", "cewas_dw")
PGHOST = os.getenv("PGHOST", "postgres")
PGPORT = os.getenv("PGPORT", "5432")

engine = create_engine(f"postgresql+psycopg2://{PGUSER}:{PGPASS}@{PGHOST}:{PGPORT}/{PGDB}", future=True)


# Read DWH data

In [ ]:
q = """
SELECT
  f.customer_id,
  f.tenure,
  f.monthly_charges,
  f.total_charges,
  f.churn,
  c.gender, c.senior_citizen, c.partner, c.dependents,
  k.contract, k.paperless_billing, k.payment_method,
  s.phone_service, s.multiple_lines, s.internet_service,
  s.online_security, s.online_backup, s.device_protection, s.tech_support,
  s.streaming_tv, s.streaming_movies
FROM dw.fact_customer_monthly_snapshot f
JOIN dw.dim_customer c USING(customer_id)
JOIN dw.dim_contract k USING(customer_id)
JOIN dw.dim_service s USING(customer_id)
"""
df = pd.read_sql(text(q), engine)
df.head(), df.shape
